# --- Day 10: Pipe Maze ---

https://adventofcode.com/2023/day/10

## Parse the Input Data

Neat trick I'm gonna try: Storing location coordinates as a complex number.  
Source: https://stackoverflow.com/questions/75793007/what-is-the-benefit-of-using-complex-numbers-to-store-graph-coordinates

In [1]:
foo = complex(1, 1)
print(foo)
print("neighbors:")
for i in range(4):
    print(foo + 1j**i)
print("Counter-clockwise rotation: right, top, left, bottom.")

(1+1j)
neighbors:
(2+1j)
(1+2j)
1j
(1+0j)
Counter-clockwise rotation: right, top, left, bottom.


OK... Actually, I found that too difficult to debug... Not gonna use it here, rn, but might try it again at a different point...

In [2]:
from collections import defaultdict

In [3]:
def parse(filename):
    """Parse input data for puzzle.

    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.

    Returns
    -------
    pipe_map : defaultdict, with a default value of "."; keys will be complex number
    representations of x, y coordinates.
    """
    # Any references that would have thrown a KeyError will create a k, v= "." pair
    pipe_map = defaultdict(lambda: ".")

    with open(f'../inputs/{filename}.txt') as f:
        for r, line in enumerate(f):
            for c, char in enumerate(line.strip()):
                if char != ".":
                    pipe_map[(r, c)] = char

    return pipe_map

In [4]:
parse("test_pipe_map0")

defaultdict(<function __main__.parse.<locals>.<lambda>()>,
            {(1, 1): 'S',
             (1, 2): '-',
             (1, 3): '7',
             (2, 1): '|',
             (2, 3): '|',
             (3, 1): 'L',
             (3, 2): '-',
             (3, 3): 'J'})

## Part 1
---

In [5]:
def build_graph(pipe_map):
    # Valid pipe connections
    valid_tops = ["|", "7", "F", "S"]
    valid_rights = ["-", "7", "J", "S"]
    valid_bottoms = ["|", "J", "L", "S"]
    valid_lefts = ["-", "F", "L", "S"]

    valid = {
        "S" : [valid_tops, valid_rights, valid_bottoms, valid_lefts],
        "|" : [valid_tops, [], valid_bottoms, []],
        "7" : [[], [], valid_bottoms, valid_lefts],
        "F" : [[], valid_rights, valid_bottoms, []],
        "J" : [valid_tops, [], [], valid_lefts],
        "L" : [valid_tops, valid_rights, [], []],
        "-" : [[], valid_rights, [], valid_lefts],
        "." : [[], [], [], []]
    }

    graph = {}

    for k in list(pipe_map.keys()):  # Convert to a list b/c defaultdict creates new dict entries
        valid_nodes = valid[pipe_map[k]]
        r, c = k
        nodes = []
        for i, deltas in enumerate([(-1, 0), (0, 1), (1, 0), (0, -1)]):
            dr, dc = deltas
            neighbor = (r + dr, c + dc)
            if pipe_map[neighbor] in valid_nodes[i]:
                if pipe_map[neighbor] == "S":
                    nodes.append("S")
                else:
                    nodes.append(neighbor)
        # Don't add any nodes that have only a single valid node:
        # It's a dud and we don't want to waste time with it.
        if len(nodes) >= 2:
             if pipe_map[k] == "S":
                k = "S"
             graph[k] = nodes

    return graph

In [6]:
build_graph(parse("test_pipe_map0"))

{'S': [(1, 2), (2, 1)],
 (1, 2): [(1, 3), 'S'],
 (1, 3): [(2, 3), (1, 2)],
 (2, 1): ['S', (3, 1)],
 (2, 3): [(1, 3), (3, 3)],
 (3, 1): [(2, 1), (3, 2)],
 (3, 2): [(3, 3), (3, 1)],
 (3, 3): [(2, 3), (3, 2)]}

In [7]:
build_graph(parse("test_pipe_map1"))

{(0, 4): [(1, 4), (0, 3)],
 'S': [(1, 2), (2, 1)],
 (1, 2): [(1, 3), 'S'],
 (1, 3): [(2, 3), (1, 2)],
 (1, 4): [(0, 4), (2, 4)],
 (2, 1): ['S', (3, 1)],
 (2, 3): [(1, 3), (3, 3)],
 (2, 4): [(1, 4), (3, 4)],
 (3, 1): [(2, 1), (3, 2)],
 (3, 2): [(3, 3), (3, 1)],
 (3, 3): [(2, 3), (3, 2)]}

In [8]:
from collections import deque

In [9]:
def solve1(graph):
    """
    Find the loop in the graph, starting at 'S' and
    return the number of steps to the furthest point in
    the loop from 'S'
    """
    step_count = 0
    visited = set()
    q = deque("S")

    while q:
        node = q.popleft()
        if node in visited:
            return int(step_count / 2)
        else:
            visited.add(node)
            next_nodes = [n for n in graph[node] if n not in visited]
            q += next_nodes
            step_count += 1


### Run on Test Data

In [10]:
solve1(build_graph(parse("test_pipe_map0"))) == 4

True

In [11]:
solve1(build_graph(parse("test_pipe_map1"))) == 4

True

In [12]:
solve1(build_graph(parse("test_pipe_map2"))) == 8

True

In [13]:
solve1(build_graph(parse("test_pipe_map3"))) == 8

True

### Run on Input Data

In [14]:
solve1(build_graph(parse("pipe_map")))

6856

## Part 2
---

My initial thought was to just get the path for the cycle and then figure out what to do next...

And then nothing really came to me.

So I read a bunch of hints on Reddit!

This thread was the most helpful: https://www.reddit.com/r/adventofcode/comments/18ey1s7/2023_day_10_part_2_stumped_on_how_to_approach_this/

Useful info/formulas:
* [Shoelace formula](https://en.wikipedia.org/wiki/Shoelace_formula)
* [Pick's theorem](https://en.wikipedia.org/wiki/Pick%27s_theorem)

In [15]:
def get_loop(graph, pipe_map):
    # First find "S" in pipe_map and swap the "S"
    # for the actual coordinates in the graph
    for i in pipe_map:
        if pipe_map[i] == "S":
            start = i
            break

    graph[i] = graph["S"]
    graph.pop("S")

    end = graph[start].pop()
    visited = set()
    q = deque([start])
    paths = [[]]

    while q:
        node = q.popleft()
        if node == "S":
            node = start

        if node not in visited:
            paths.append(paths[-1] + [node])

            if node == end:
                return paths[-1]
            else:
                visited.add(node)
                q += graph[node]


In [16]:
get_loop(build_graph(parse("test_pipe_map4")), parse("test_pipe_map4"))

[(1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (2, 9),
 (3, 9),
 (4, 9),
 (5, 9),
 (6, 9),
 (7, 9),
 (7, 8),
 (7, 7),
 (7, 6),
 (6, 6),
 (5, 6),
 (5, 7),
 (5, 8),
 (4, 8),
 (3, 8),
 (2, 8),
 (2, 7),
 (2, 6),
 (2, 5),
 (2, 4),
 (2, 3),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 2),
 (5, 3),
 (5, 4),
 (6, 4),
 (7, 4),
 (7, 3),
 (7, 2),
 (7, 1),
 (6, 1),
 (5, 1),
 (4, 1),
 (3, 1),
 (2, 1)]

In [17]:
def solve2(loop):

    # First use shoelace formula to calculate the area, A, of a "simple" (non-overlapping) polygon.
    temp = sum([(loop[i][0] * loop[i+1][1]) - (loop[i][1] * loop[i+1][0]) for i in range(len(loop) - 1)])
    temp += (loop[-1][0] * loop[0][1]) - (loop[-1][1] * loop[0][0])
    area = temp / 2

    # Then use Pick's theorem to calculate the number of interior points, given the area and the number
    # of nodes on the simple polygon.
    num_nodes = len(loop)
    num_interior_points = abs(area) - (num_nodes / 2) + 1

    return int(num_interior_points)

### Run on Test Data

In [18]:
solve2(get_loop(build_graph(parse("test_pipe_map4")), parse("test_pipe_map4"))) == 4

True

In [19]:
solve2(get_loop(build_graph(parse("test_pipe_map5")), parse("test_pipe_map5"))) == 8

True

In [20]:
solve2(get_loop(build_graph(parse("test_pipe_map6")), parse("test_pipe_map6"))) == 10

True

In [21]:
solve2(get_loop(build_graph(parse("test_pipe_map7")), parse("test_pipe_map7"))) == 4

True

### Run on Input Data

In [22]:
solve2(get_loop(build_graph(parse("pipe_map")), parse("pipe_map")))

501